In [ ]:
# ENV["PYTHON"]= "/kuacc/users/oulas15/.conda/envs/oulas15/bin/python"
# using Pkg
# Pkg.build("PyCall")

In [1]:
using Knet, CUDA, MLDatasets, ArgParse, Images, Random, Statistics,Base, Plots,NPZ
#include(Knet.dir("data","imagenet.jl"))
const F = Float32
Atype = CUDA.functional() ? KnetArray{F} : Array{F}

KnetArray{Float32, N} where N

In [2]:
train_data = npzread("../Mouse_n20/train/train_data.npz")

Dict{String, Array{T, 3} where T} with 4 entries:
  "X_val"   => [15.1078 0.766439 … 20.7861 -12.73; -10.8562 -2.43127 … 28.4208 …
  "Y_val"   => UInt16[0x0000 0x0000 … 0x0000 0x0000; 0x0000 0x0000 … 0x0000 0x0…
  "X_train" => [-8.65205 -38.3173 … -9.87815 3.37247; -35.7896 -8.26069 … -16.3…
  "Y_train" => UInt16[0x0000 0x0000 … 0x0000 0x0000; 0x0000 0x0000 … 0x0000 0x0…

In [3]:
xtrn=train_data["X_train"];
xval=train_data["X_val"];
xtrn=permutedims(xtrn, [2, 3, 1]);
xval=permutedims(xval, [2, 3, 1]);
println(size(xtrn))
println(size(xval))

(128, 128, 908)
(128, 128, 160)


In [4]:
l90 = [rotl90(xtrn[:,:,i],1) for i in 1:908]
l180 = [rotl90(xtrn[:,:,i],2) for i in 1:908]
r90 = [rotl90(xtrn[:,:,i],3) for i in 1:908]
l90 = cat(l90...,dims=3);
l180 = cat(l180...,dims=3);
r90 = cat(r90...,dims=3);
rotated_matrix = cat([xtrn,l90,l180,r90]...,dims=3);
augmented = cat([rotated_matrix[end:-1:1,:,:],rotated_matrix]...,dims=3);

In [5]:
data = cat(xtrn,xval,dims=3);

In [6]:
data_mean = mean(data);
data_std = stdm(data,data_mean);
g_std = 20;

In [16]:
batch_size = 32
dtrn = minibatch(augmented, batch_size; xsize = (128, 128, 1,:), xtype = Atype,shuffle=true)
dval = minibatch(xval, 32; xsize = (128, 128, 1,:), xtype = Atype);

In [17]:
function encode(ϕ, x)
    x = conv4(ϕ[1], x, padding=1)
    x = relu.(x .+ ϕ[2])
    x = conv4(ϕ[3], x, padding=1, stride=1)
    x = relu.(x .+ ϕ[4])
    x = pool(x, stride = 2, mode=0)

    x = conv4(ϕ[5], x, padding=1)
    x = relu.(x .+ ϕ[6])
    x = conv4(ϕ[7], x, padding=1, stride=1)
    x = relu.(x .+ ϕ[8])
    x = pool(x, stride = 2, mode=0)

    x = conv4(ϕ[9], x, padding=1)
    x = relu.(x .+ ϕ[10])
    x = conv4(ϕ[11], x, padding=1, stride=1)
    x = relu.(x .+ ϕ[12])
    x = pool(x, stride = 2, mode=0)

    
    mu = conv4(ϕ[13], x, padding=1, stride=1)
    mu = mu .+ ϕ[14]
    
    logvar = conv4(ϕ[15], x, padding=1, stride=1)
    logvar = logvar .+ ϕ[16]
    
    return mu, logvar
end

function decode(θ, z)
    z = conv4(θ[1], z, padding=1)
    z = relu.(z .+ θ[2])
    z = conv4(θ[3], z, padding=1, stride=1)
    z = relu.(z .+ θ[4])
    z = deconv4(θ[5], z, padding=0, stride=2)
    z = relu.(z .+ θ[6])


    z = conv4(θ[7], z, padding=1)
    z = relu.(z .+ θ[8])
    z = conv4(θ[9], z, padding=1, stride=1)
    z = relu.(z .+ θ[10]) 
    z = deconv4(θ[11], z, padding=0, stride=2)
    z = relu.(z .+ θ[12])
    

    z = conv4(θ[13], z, padding=1)
    z = relu.(z .+ θ[14])
    z = conv4(θ[15], z, padding=1, stride=1)
    z = relu.(z .+ θ[16]) 
    z = deconv4(θ[17], z, padding=0, stride=2)
    z = relu.(z .+ θ[18])

    z = conv4(θ[19], z, padding=1)
    z = z .+ θ[20]
    return z
end



function weights(channel)
    
    θ = [] # z->x

    push!(θ, xavier(3, 3, channel*2, channel*4))
    push!(θ, zeros(1, 1, channel*4, 1))

    push!(θ, xavier(3,3,channel*4,channel*4))
    push!(θ, zeros(1, 1, channel*4, 1))
    
    push!(θ, xavier(2, 2, channel*4, channel*4))
    push!(θ, zeros(1,1,channel*4,1))
    
    
    push!(θ, xavier(3, 3, channel*4, channel*2))
    push!(θ, zeros(1,1, channel*2,1))
    
    push!(θ, xavier(3,3,channel*2,channel*2))
    push!(θ, zeros(1, 1, channel*2, 1))
    
    push!(θ, xavier(2,2,channel*2,channel*2))
    push!(θ, zeros(1, 1, channel*2, 1))
    
    
    push!(θ, xavier(3, 3, channel*2, channel))
    push!(θ, zeros(1,1, channel,1))
    
    push!(θ, xavier(3,3,channel,channel))
    push!(θ, zeros(1, 1, channel, 1))
    
    push!(θ, xavier(2,2,channel,channel))
    push!(θ, zeros(1, 1, channel, 1))
    

    push!(θ, xavier(3,3,channel,1))
    push!(θ, zeros(1, 1, 1, 1))
    
    
    θ = map(a->convert(Atype,a), θ)

    ϕ = [] # x->z

    push!(ϕ, xavier(3, 3, 1, channel))
    push!(ϕ, zeros(1, 1, channel, 1))

    push!(ϕ, xavier(3, 3, channel, channel))
    push!(ϕ, zeros(1, 1, channel, 1))
    
    
    push!(ϕ, xavier(3, 3, channel, channel*2))
    push!(ϕ, zeros(1,1, channel*2, 1))

    push!(ϕ, xavier(3, 3, channel*2,channel*2))
    push!(ϕ, zeros(1, 1, channel*2, 1))
    
    
    push!(ϕ, xavier(3, 3, channel*2, channel*4))
    push!(ϕ, zeros(1,1, channel*4, 1))

    push!(ϕ, xavier(3, 3, channel*4, channel*4))
    push!(ϕ, zeros(1, 1, channel*4, 1))
    
    
    push!(ϕ, xavier(3, 3, channel*4, channel*2))
    push!(ϕ, zeros(1, 1, channel*2, 1))
    
    push!(ϕ, xavier(3, 3, channel*4, channel*2))
    push!(ϕ, zeros(1, 1, channel*2, 1))

    ϕ = map(a->convert(Atype,a), ϕ)
    return θ, ϕ
end

weights (generic function with 1 method)

In [18]:
function KLD(mu, logvar)
    kl_error = -0.5 * sum(1 .+ (logvar - mu.*mu - exp.(logvar)))
    return kl_error
end

KLD (generic function with 1 method)

In [19]:
function MSE(x, y,gaus_std,data_std)
    return mean((x - y).^2) / (2.0*(gaus_std/data_std)^2)
end

MSE (generic function with 1 method)

In [20]:
function reparameterize(mu,logvar)
    std = exp.(logvar.*0.5)
    epsilon = convert(Atype, randn(F, size(mu)))
    z = mu .+ (epsilon.*std)
    return z 
end

reparameterize (generic function with 1 method)

In [39]:
wdec, wenc = weights(32);
w = Dict(
        :encoder => wenc,
        :decoder => wdec)
opt = Dict(
        :encoder => map(wi->Knet.Adam(lr = 1e-3), w[:encoder]),
        :decoder => map(wi->Knet.Adam(lr = 1e-3), w[:decoder]),
    )

Dict{Symbol, Vector{Adam}} with 2 entries:
  :decoder => [Adam(0.001, 0.9, 0.999, 1.0e-8, 0, 0.0, nothing, nothing), Adam(…
  :encoder => [Adam(0.001, 0.9, 0.999, 1.0e-8, 0, 0.0, nothing, nothing), Adam(…

In [40]:
opt

Dict{Symbol, Vector{Adam}} with 2 entries:
  :decoder => [Adam(0.001, 0.9, 0.999, 1.0e-8, 0, 0.0, nothing, nothing), Adam(…
  :encoder => [Adam(0.001, 0.9, 0.999, 1.0e-8, 0, 0.0, nothing, nothing), Adam(…

In [41]:
function loss(w,x,gaus_std,data_mean,data_std,epoch)
    wdec, wenc = w[:decoder], w[:encoder]
    x_normalized = (x .- data_mean)./ data_std
    mu, logvar = encode(wenc, x_normalized)
    z = reparameterize(mu,logvar)
    y = decode(wdec,z)
    if epoch <= 5
        return MSE(x_normalized,y,gaus_std,data_std) + ((epoch-1)/5)*(KLD(mu,logvar)/(128*128*32))
    else
        return MSE(x_normalized,y,gaus_std,data_std) + (KLD(mu,logvar)/(128*128*32))
    end
end
lossgradient = grad(loss);

In [42]:
function val_mse(w, x,gaus_std,data_mean,data_std)
    wdec, wenc = w[:decoder], w[:encoder]
    x_normalized = (x .- data_mean)./ data_std
    mu, logvar = encode(wenc, x_normalized)
    z = reparameterize(mu,logvar)
    y = decode(wdec,z)
    return MSE(x_normalized,y,gaus_std,data_std), KLD(mu,logvar)/(128*128*32)
end

val_mse (generic function with 1 method)

In [ ]:
number_of_epochs = 3000
val_total_loss = []
val_recon_loss = []
val_kld_loss = []
best_val_loss = 10^10
best_val_loss_epoch = 0
val_len = length(dval)
for epoch in 1:number_of_epochs
    for (i,x)  in enumerate(dtrn)
        dw = lossgradient(w, x,g_std,data_mean,data_std,epoch)
        update!(w, dw, opt)
    end
    mse_sum = 0
    kld_sum = 0
    
    for (j,x) in enumerate(dval)
        mse,kld  = val_mse(w,x,g_std,data_mean,data_std) 
        mse_sum += mse
        kld_sum += kld
    end
    print("Epoch: ",epoch)
    mse_sum = mse_sum/val_len
    kld_sum = kld_sum/val_len
    total_loss = mse_sum + kld_sum
    print(", MSE : ",mse_sum,", KLD : ",kld_sum)
    if total_loss<best_val_loss
        best_val_loss = total_loss
        best_val_loss_epoch = epoch
        print(", !!New Best!!")
        Knet.save("best_aug.jld2","w",w)
    end
    
    if (epoch - best_val_loss_epoch)>30
        for layer in opt[:encoder]
            layer.lr *= 0.5
        end
        for layer in opt[:decoder]
            layer.lr *= 0.5
        end 
        print(", Reduced lr")
        best_val_loss_epoch = epoch
    end

    println("")
    Knet.save("last_aug.jld2","w",w)
    push!(val_total_loss,total_loss)
    push!(val_recon_loss,mse_sum)
    push!(val_kld_loss,kld_sum)  
end

Epoch: 1, MSE : 0.6810411524078797, KLD : 7.381231784820557, !!New Best!!
Epoch: 2, MSE : 0.6458994624667954, KLD : 0.2006791353225708, !!New Best!!
Epoch: 3, MSE : 1.0356020094904754, KLD : 0.17307038009166717
Epoch: 4, MSE : 0.6087963614545189, KLD : 0.09360297173261642, !!New Best!!
Epoch: 5, MSE : 0.6162447404248325, KLD : 0.0704823762178421, !!New Best!!
Epoch: 6, MSE : 0.5899463350587578, KLD : 0.0615057572722435, !!New Best!!
Epoch: 7, MSE : 0.6177980259227447, KLD : 0.06148519441485405
Epoch: 8, MSE : 0.5755187765449611, KLD : 0.05879476889967918, !!New Best!!
Epoch: 9, MSE : 0.5705900984406418, KLD : 0.058967381715774536, !!New Best!!
Epoch: 10, MSE : 0.5737660395075117, KLD : 0.05747601240873337
Epoch: 11, MSE : 0.5653018437445494, KLD : 0.056487925350666046, !!New Best!!
Epoch: 12, MSE : 0.5730658817086827, KLD : 0.05370569676160812
Epoch: 13, MSE : 0.5645572179514604, KLD : 0.049896573275327684, !!New Best!!
Epoch: 14, MSE : 0.5550717551299866, KLD : 0.04952164515852928, !!